# Generate Graph Data for MODEl input

In [1]:
from DOLPHIN import get_graph_input

/mnt/md0/kailu/anaconda3/envs/DOLPHIN/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
metadata = "your_metaData.csv"
df_label = pd.read_csv(metadata, sep='\t')
total_sample_size = len(df_label)
main_folder="Path_to_save_output_data"

# Define your study name (update this for your specific study)
study_name = "STUDY"

gnn_run_num = 100

output_directory = os.path.join(main_folder, "final_data")
graph_directory = os.path.join(main_folder "06_graph_mtx")
exon_gene_featurecount_directory=os.path.join(main_folder, "04_exon_gene_cnt")

# Required files
gene_annotation = "gene_id_annotation.csv"
gtf_pkl_path = "gtf.pkl"
gtf_jun_pkl_path = "df_jun_gtf.pkl"
gtf_path= "Homo_sapiens.GRCh38.107.gtf"

In [ ]:
### Optional: Define Cell Type Information in Metadata
celltypename="cluster"
# please assign value to each cell types
celltype_mapper = {'celltype1':0, 'celltype2':1, 'celltype3':2, 'celltype4':3, ...}
 

In [ ]:
print("Start Construct Data Input for Pytorch Geometric")
with tqdm(total=total_sample_size) as pbar_gnn:
    for i in range(0, total_sample_size):
        if i%gnn_run_num==0:
            pbar_gnn= get_graph_input(pbar_gnn, i, gnn_run_num, output_directory, study_name, celltypename=celltypename, mapper=celltype_mapper)

#### combine all  geometric .pt files
total_number_gnn_anndata = math.ceil(total_sample_size/gnn_run_num)
for _idx, _gnn_idx in enumerate(range(0, total_number_gnn_anndata)):
    _temp_gnn = torch.load(os.path.join(output_directory, "geometric_"+study_name+"_"+str(_gnn_idx)+".pt"))
    if _idx ==0:
        combine_gnn = _temp_gnn
    else:
        combine_gnn += _temp_gnn
torch.save(combine_gnn, os.path.join(output_directory, "geometric_"+study_name+".pt"))


In [ ]:
### Remove All Temporary Datasets
file_pattern = os.path.join(output_directory, f"geometric_{study_name}_*")

files_to_remove = glob.glob(file_pattern)

# Remove each file
for file in files_to_remove:
    if os.path.exists(file):  # Check if the file exists
        os.remove(file)
        print(f"Removed: {file}")
    else:
        print(f"File not found: {file}")